In [1]:
# load packages

import pandas as pd
import numpy as np
np.random.seed(59)
import seaborn as sns
import math
import string

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

import urllib.request

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# read in list of words to be used as database of answers

url = "https://www-cs-faculty.stanford.edu/~knuth/sgb-words.txt"
file = urllib.request.urlopen(url)
db = []

for line in file:
    decoded_line = line.decode("utf-8")
    db.append(decoded_line)
    
db

['which\n',
 'there\n',
 'their\n',
 'about\n',
 'would\n',
 'these\n',
 'other\n',
 'words\n',
 'could\n',
 'write\n',
 'first\n',
 'water\n',
 'after\n',
 'where\n',
 'right\n',
 'think\n',
 'three\n',
 'years\n',
 'place\n',
 'sound\n',
 'great\n',
 'again\n',
 'still\n',
 'every\n',
 'small\n',
 'found\n',
 'those\n',
 'never\n',
 'under\n',
 'might\n',
 'while\n',
 'house\n',
 'world\n',
 'below\n',
 'asked\n',
 'going\n',
 'large\n',
 'until\n',
 'along\n',
 'shall\n',
 'being\n',
 'often\n',
 'earth\n',
 'began\n',
 'since\n',
 'study\n',
 'night\n',
 'light\n',
 'above\n',
 'paper\n',
 'parts\n',
 'young\n',
 'story\n',
 'point\n',
 'times\n',
 'heard\n',
 'whole\n',
 'white\n',
 'given\n',
 'means\n',
 'music\n',
 'miles\n',
 'thing\n',
 'today\n',
 'later\n',
 'using\n',
 'money\n',
 'lines\n',
 'order\n',
 'group\n',
 'among\n',
 'learn\n',
 'known\n',
 'space\n',
 'table\n',
 'early\n',
 'trees\n',
 'short\n',
 'hands\n',
 'state\n',
 'black\n',
 'shown\n',
 'stood\n',
 'fr

In [3]:
# cleaning up list, converting to upper case to match wordle

db_temp = []

for x in db:
    cut_string = x[:5]
    db_temp.append(cut_string.upper())
    
db_temp 
db = db_temp.copy()
db_temp.clear()

db

['WHICH',
 'THERE',
 'THEIR',
 'ABOUT',
 'WOULD',
 'THESE',
 'OTHER',
 'WORDS',
 'COULD',
 'WRITE',
 'FIRST',
 'WATER',
 'AFTER',
 'WHERE',
 'RIGHT',
 'THINK',
 'THREE',
 'YEARS',
 'PLACE',
 'SOUND',
 'GREAT',
 'AGAIN',
 'STILL',
 'EVERY',
 'SMALL',
 'FOUND',
 'THOSE',
 'NEVER',
 'UNDER',
 'MIGHT',
 'WHILE',
 'HOUSE',
 'WORLD',
 'BELOW',
 'ASKED',
 'GOING',
 'LARGE',
 'UNTIL',
 'ALONG',
 'SHALL',
 'BEING',
 'OFTEN',
 'EARTH',
 'BEGAN',
 'SINCE',
 'STUDY',
 'NIGHT',
 'LIGHT',
 'ABOVE',
 'PAPER',
 'PARTS',
 'YOUNG',
 'STORY',
 'POINT',
 'TIMES',
 'HEARD',
 'WHOLE',
 'WHITE',
 'GIVEN',
 'MEANS',
 'MUSIC',
 'MILES',
 'THING',
 'TODAY',
 'LATER',
 'USING',
 'MONEY',
 'LINES',
 'ORDER',
 'GROUP',
 'AMONG',
 'LEARN',
 'KNOWN',
 'SPACE',
 'TABLE',
 'EARLY',
 'TREES',
 'SHORT',
 'HANDS',
 'STATE',
 'BLACK',
 'SHOWN',
 'STOOD',
 'FRONT',
 'VOICE',
 'KINDS',
 'MAKES',
 'COMES',
 'CLOSE',
 'POWER',
 'LIVED',
 'VOWEL',
 'TAKEN',
 'BUILT',
 'HEART',
 'READY',
 'QUITE',
 'CLASS',
 'BRING',
 'ROUND',


In [4]:
# checking length of list

len(db)

5757

In [5]:
# function for splitting words

def split(word):
    return [char for char in word]

In [6]:
# function for determing distribution of letters, based upon original data set. Also determined letter weighting

char_count = {}
char_totals = {}
char_st = {}
char_temp = []

def char_totals_func(word_list, reweight = True):

    char_count.clear()
    
    for word in word_list:
        char_temp = split(word)

        for char in char_temp:
            if (char in char_count):
                char_count[char] += 1
            else:
                char_count[char] = 1
                
    char_totals.clear()
    
    sorted_chars = dict(sorted(char_count.items(), key = lambda x: x[0].lower()))
    
    for k,v in sorted_chars.items():
        char_totals[k] = v
    
    if len(char_st) == 0 or reweight == True:
        
        char_st.clear()

        for k,v in char_totals.items():
            char_st[k] = v/sum(char_totals.values())

        char_max = (max(char_st.values()))

        for k,v in char_st.items():
            char_st[k] = (v/char_max)*1

In [7]:
# function for sorting data set to only words in five unique values

db_uniques = []

def db_uniques_func(word_list):

    db_uniques.clear()
    
    for word in word_list:
        char_temp = split(word)

        if (len(set(char_temp)) == len(word)): db_uniques.append(word)

In [8]:
# function for weighting words in list. Gives highest strength word in list
# db stores words

word_st = {}
char_st_temp = {}

def word_st_func(word_list):
    
    word_st.clear()
    
    for word in word_list:
        char_temp = split(word)

        char_st_temp = {}
        word_st_temp = 1

        for char in char_temp:
            if (char in char_st_temp):
                char_st_temp[char] = char_st_temp[char] * char_st[char]

            else:
                char_st_temp[char] = char_st[char]

        for char in char_st_temp:
            word_st_temp = word_st_temp * char_st_temp[char]

        word_st[word] = word_st_temp

In [9]:
# creating list of all possible letters per slot

char_loc_ref = {}

def char_loc_ref_full():

    char_loc_ref.clear()

    for i in range(5):

        char_loc_ref[i] = "A"
        char_loc_ref[i] = [char_loc_ref[i], "B"]

        for char in string.ascii_uppercase[2:]:
            char_loc_ref[i].append(char)

In [23]:
# function for filtering possible characters for each slot based upon result
   
char_include = []
    
def char_loc_ref_filt_func(word_played_rX, word_played_rX_result):

    char_include.clear()
    char_temp = split(word_played_rX)

    for i in range(len(word_played_rX_result)):
        
        if word_played_rX_result[i] == 0:
            if char_temp[i] in char_include and char_temp[i] in char_loc_ref[i]:
                char_loc_ref[i].remove(char_temp[i])
                
            else:
                
                for slot in char_loc_ref:
                    if char_temp[i] in char_loc_ref[slot] and len(char_loc_ref[slot]) != 1:
                        char_loc_ref[slot].remove(char_temp[i])

        if word_played_rX_result[i] == 1:
            # Kept to account for answer not being in data set, leading to error
            if char_temp[i] in char_loc_ref[i]: char_loc_ref[i].remove(char_temp[i])

            if char_temp[i] not in char_include: char_include.append(char_temp[i])

        if word_played_rX_result[i] == 2:
                char_loc_ref[i].clear()
                char_loc_ref[i].append(char_temp[i])

In [11]:
# function for filtering data set based upon possible character values

def db_filtered_func(db_to_filt):
    
    global db_filtered    
    db_filtered = db_to_filt.copy()

    for word in db_to_filt:
        char_temp = split(word)

        for i in range(len(char_temp)):
            if char_temp[i] not in char_loc_ref[i]:
                db_filtered.remove(word)            
                break   
    
    # check for answer not in data set
    if len(db_filtered) == 0:
        print("Answer not in word list")

# Below starts wordle solver

# Solver for full list

In [26]:
# establishing loop through database, testing on highest strength word
answer_rewt_history = pd.DataFrame(columns = ['answer', 'first_guess', 'second_guess', 'third_guess', 
                                   'fourth_guess', 'fifth_guess', 'sixth_guess', 'No Solution'])

answer_word = []
answer_char = []
word_played = []
word_played_char = []
word_played_result = []
result_char_count_temp = {}
answer_char_count_temp = {}

for word in db:
    print("Word Guessing: " + str(word))
    
    char_loc_ref_full()

    answer_char.clear()
    word_played_char.clear()
    word_played_result.clear()
    result_char_count_temp.clear()
    answer_char_count_temp.clear()
    
    # establishing round counter
    rounds_played = 0
    
    answer_word = word
    answer_char = split(answer_word)
    
    for char in answer_char:
        if (char in answer_char_count_temp):
            answer_char_count_temp[char] += 1
        else:
            answer_char_count_temp[char] = 1
            
    answer_rewt_history.loc[len(answer_rewt_history.index), 'answer'] = answer_word
    
    # running counter on original database           
    char_totals_func(db)

    # sorting for uniques on original data set
    db_uniques_func(db)
    word_st_func(db_uniques)

    while rounds_played <= 6:

        # determining word to play, splitting for characters, and adding to data set of answers
        word_played = max(word_st, key=word_st.get)
        word_played_char = split(word_played)
        word_played_result.clear()
        result_char_count_temp.clear()
        rounds_played += 1
        answer_rewt_history.iloc[len(answer_rewt_history.index)-1, [rounds_played]] = word_played

        # finding result from playing word
        for i in range(len(word_played)):
            if word_played_char[i] not in answer_char:
                word_played_result.append(0)

            elif word_played_char[i] == answer_char[i]:
                word_played_result.append(2)

                if (word_played_char[i] in result_char_count_temp):
                    result_char_count_temp[word_played_char[i]] += 1
                else:
                    result_char_count_temp[word_played_char[i]] = 1

            # do not need to check that it does not match as all matching letters will be handled above
            elif word_played_char[i] in answer_char:

                if word_played_char[i] in result_char_count_temp:
                    if result_char_count_temp[word_played_char[i]] == answer_char_count_temp[word_played_char[i]]:
                        word_played_result.append(0)
                        
                    else:
                        word_played_result.append(1) 
                        result_char_count_temp[word_played_char[i]] += 1

                else:
                    word_played_result.append(1)                   
                    result_char_count_temp[word_played_char[i]] = 1                    
            else:
                print("Error in assigning values to word_played_result based upon word played")
                break

        print(word_played, word_played_result)

        if word_played_result == [2,2,2,2,2]:
            # fill unfilled round columns with placeholder (0?)
            break

        if rounds_played == 6:
            answer_rewt_history.loc[len(answer_rewt_history.index)-1, 'No Solution'] = 1
            break

        else:

            # running filtering character reference list based on first round
            char_loc_ref_filt_func(word_played, word_played_result)

            # filtered data set based upon results of round
            if rounds_played == 1:
                db_filtered_func(db)
            else:
                db_filtered_func(db_filtered)

        # determing distribution of letters, based upon filtered data set, if reweight = 'True' determining new letter weighting

        char_totals_func(db_filtered, reweight = True)

        # filtering for words with five unique letters on filtered data set for first three rounds
        if rounds_played <= 3 and len(db_filtered) != 1:
            db_uniques_func(db_filtered)
            
            if bool(db_uniques):
                word_st_func(db_uniques)
            
            else:
                word_st_func(db_filtered)

        else:
            word_st_func(db_filtered)

Word Guessing: WHICH
AROSE [0, 0, 0, 0, 0]
UNITY [0, 0, 2, 0, 0]
FLICK [0, 0, 2, 2, 0]
WHICH [2, 2, 2, 2, 2]
Word Guessing: THERE
AROSE [0, 1, 0, 0, 2]
GUILE [0, 0, 0, 0, 2]
RHYME [1, 2, 0, 0, 2]
WHEEE [0, 2, 2, 1, 2]
THERE [2, 2, 2, 2, 2]
Word Guessing: THEIR
AROSE [0, 1, 0, 0, 1]
UTERI [0, 1, 2, 1, 1]
DWELT [0, 0, 2, 0, 1]
THEIR [2, 2, 2, 2, 2]
Word Guessing: ABOUT
AROSE [2, 0, 2, 0, 0]
ALOFT [2, 0, 2, 0, 2]
ABOUT [2, 2, 2, 2, 2]
Word Guessing: WOULD
AROSE [0, 0, 1, 0, 0]
UNITY [1, 0, 0, 0, 0]
MUCHO [0, 1, 0, 0, 1]
WOULD [2, 2, 2, 2, 2]
Word Guessing: THESE
AROSE [0, 0, 0, 2, 2]
DULSE [0, 0, 0, 2, 2]
SENSE [1, 1, 0, 2, 2]
THESE [2, 2, 2, 2, 2]
Word Guessing: OTHER
AROSE [0, 1, 1, 0, 1]
OILER [2, 0, 0, 2, 2]
OTHER [2, 2, 2, 2, 2]
Word Guessing: WORDS
AROSE [0, 1, 1, 1, 0]
LOUIS [0, 2, 0, 0, 2]
CORNS [0, 2, 2, 0, 2]
FORTS [0, 2, 2, 0, 2]
WORDS [2, 2, 2, 2, 2]
Word Guessing: COULD
AROSE [0, 0, 1, 0, 0]
UNITY [1, 0, 0, 0, 0]
MUCHO [0, 1, 1, 0, 1]
COULD [2, 2, 2, 2, 2]
Word Guessing: WRIT

AROSE [0, 2, 0, 1, 1]
TRIES [2, 2, 0, 2, 2]
TRUES [2, 2, 0, 2, 2]
TREES [2, 2, 2, 2, 2]
Word Guessing: SHORT
AROSE [0, 1, 2, 1, 0]
SLOTH [2, 0, 2, 1, 1]
SNORT [2, 0, 2, 2, 2]
SHORT [2, 2, 2, 2, 2]
Word Guessing: HANDS
AROSE [1, 0, 0, 1, 0]
TAILS [0, 2, 0, 0, 2]
PANGS [0, 2, 2, 0, 2]
BANDS [0, 2, 2, 2, 2]
SANDS [1, 2, 2, 2, 2]
HANDS [2, 2, 2, 2, 2]
Word Guessing: STATE
AROSE [1, 0, 0, 1, 2]
STALE [2, 2, 2, 0, 2]
STAGE [2, 2, 2, 0, 2]
STAKE [2, 2, 2, 0, 2]
STATE [2, 2, 2, 2, 2]
Word Guessing: BLACK
AROSE [1, 0, 0, 0, 0]
INLAY [0, 0, 1, 1, 0]
MATCH [0, 1, 0, 2, 0]
BLACK [2, 2, 2, 2, 2]
Word Guessing: SHOWN
AROSE [0, 0, 2, 1, 0]
PLOTS [0, 0, 2, 0, 1]
SMOKY [2, 0, 2, 0, 0]
SHOWN [2, 2, 2, 2, 2]
Word Guessing: STOOD
AROSE [0, 0, 2, 1, 0]
PLOTS [0, 0, 2, 1, 1]
STONY [2, 2, 2, 0, 0]
STOCK [2, 2, 2, 0, 0]
STOOD [2, 2, 2, 2, 2]
Word Guessing: FRONT
AROSE [0, 2, 2, 0, 0]
PRONG [0, 2, 2, 2, 0]
CRONY [0, 2, 2, 2, 0]
FRONT [2, 2, 2, 2, 2]
Word Guessing: VOICE
AROSE [0, 0, 1, 0, 2]
GUILE [0, 0, 2, 0,

UNITY [1, 0, 0, 2, 0]
MOUTH [2, 2, 2, 2, 2]
Word Guessing: SOLVE
AROSE [0, 0, 1, 1, 2]
GUILE [0, 0, 0, 1, 2]
MONTE [0, 2, 0, 0, 2]
DOLCE [0, 2, 2, 0, 2]
SOLVE [2, 2, 2, 2, 2]
Word Guessing: NORTH
AROSE [0, 1, 1, 0, 0]
UNITY [0, 1, 0, 2, 0]
NORTH [2, 2, 2, 2, 2]
Word Guessing: VALUE
AROSE [1, 0, 0, 0, 2]
GUILE [0, 1, 0, 1, 2]
VALUE [2, 2, 2, 2, 2]
Word Guessing: DEATH
AROSE [1, 0, 0, 0, 1]
IDEAL [0, 1, 1, 1, 0]
YENTA [0, 2, 0, 2, 1]
DEATH [2, 2, 2, 2, 2]
Word Guessing: MAYBE
AROSE [1, 0, 0, 0, 2]
GUILE [0, 0, 0, 0, 2]
THANE [0, 0, 1, 0, 2]
MAYBE [2, 2, 2, 2, 2]
Word Guessing: HAPPY
AROSE [1, 0, 0, 0, 0]
INLAY [0, 0, 0, 1, 2]
DUMPY [0, 0, 0, 2, 2]
PAPPY [1, 2, 2, 2, 2]
HAPPY [2, 2, 2, 2, 2]
Word Guessing: TELLS
AROSE [0, 0, 0, 1, 1]
ISLET [0, 1, 2, 1, 1]
YELPS [0, 2, 2, 0, 2]
MELTS [0, 2, 2, 1, 2]
SELLS [1, 2, 2, 2, 2]
TELLS [2, 2, 2, 2, 2]
Word Guessing: GIVES
AROSE [0, 0, 0, 1, 1]
ISLET [1, 1, 0, 2, 0]
DINES [0, 2, 0, 2, 2]
PIKES [0, 2, 0, 2, 2]
VISES [1, 2, 1, 2, 2]
SIXES [1, 2, 0, 2,

ISLET [0, 1, 0, 1, 2]
SPENT [2, 2, 2, 2, 2]
Word Guessing: LOCAL
AROSE [1, 0, 1, 0, 0]
INLAY [0, 0, 1, 2, 0]
OCTAL [1, 1, 0, 2, 2]
FOCAL [0, 2, 2, 2, 2]
LOCAL [2, 2, 2, 2, 2]
Word Guessing: WAXES
AROSE [1, 0, 0, 1, 1]
TALES [0, 2, 0, 2, 2]
CAVES [0, 2, 0, 2, 2]
GAMES [0, 2, 0, 2, 2]
EASES [1, 2, 1, 2, 2]
SAKES [1, 2, 0, 2, 2]
Word Guessing: KNOWS
AROSE [0, 0, 2, 1, 0]
PLOTS [0, 0, 2, 0, 2]
KNOBS [2, 2, 2, 0, 2]
KNOWS [2, 2, 2, 2, 2]
Word Guessing: TRAIN
AROSE [1, 2, 0, 0, 0]
TRAIN [2, 2, 2, 2, 2]
Word Guessing: BREAD
AROSE [1, 2, 0, 0, 1]
TREAD [0, 2, 2, 2, 2]
BREAD [2, 2, 2, 2, 2]
Word Guessing: HOMES
AROSE [0, 0, 1, 1, 1]
ISLET [0, 1, 0, 2, 0]
NOSED [0, 2, 1, 2, 0]
POKES [0, 2, 0, 2, 2]
COMES [0, 2, 2, 2, 2]
HOMES [2, 2, 2, 2, 2]
Word Guessing: TEETH
AROSE [0, 0, 0, 0, 1]
YIELD [0, 0, 2, 0, 0]
TEETH [2, 2, 2, 2, 2]
Word Guessing: COAST
AROSE [1, 0, 1, 2, 0]
LOTSA [0, 2, 1, 2, 1]
MOIST [0, 2, 0, 2, 2]
COAST [2, 2, 2, 2, 2]
Word Guessing: THICK
AROSE [0, 0, 0, 0, 0]
UNITY [0, 0, 2, 1, 

AROSE [0, 1, 0, 1, 2]
GUILE [0, 0, 0, 0, 2]
RHYME [1, 0, 0, 0, 2]
PEEVE [0, 2, 1, 2, 2]
VERVE [1, 2, 2, 2, 2]
SERVE [2, 2, 2, 2, 2]
Word Guessing: ANGRY
AROSE [2, 1, 0, 0, 0]
ALIGN [2, 0, 0, 1, 1]
AUGHT [2, 0, 2, 0, 0]
ANGRY [2, 2, 2, 2, 2]
Word Guessing: CROWD
AROSE [0, 2, 2, 0, 0]
PRONG [0, 2, 2, 0, 0]
BROTH [0, 2, 2, 0, 0]
CROWD [2, 2, 2, 2, 2]
Word Guessing: WHEEL
AROSE [0, 0, 0, 0, 1]
YIELD [0, 0, 2, 1, 0]
CLEFT [0, 1, 2, 0, 0]
KNEEL [0, 0, 2, 2, 2]
WHEEL [2, 2, 2, 2, 2]
Word Guessing: QUICK
AROSE [0, 0, 0, 0, 0]
UNITY [1, 0, 2, 0, 0]
FLICK [0, 0, 2, 2, 2]
QUICK [2, 2, 2, 2, 2]
Word Guessing: DRESS
AROSE [0, 2, 0, 2, 1]
CREST [0, 2, 2, 2, 0]
FRESH [0, 2, 2, 2, 0]
DRESS [2, 2, 2, 2, 2]
Word Guessing: SHARE
AROSE [1, 1, 0, 1, 2]
STALE [2, 0, 2, 0, 2]
SHAPE [2, 2, 2, 0, 2]
SHARE [2, 2, 2, 2, 2]
Word Guessing: ALIVE
AROSE [2, 0, 0, 0, 2]
AGILE [2, 0, 2, 1, 2]
AMIDE [2, 0, 2, 0, 2]
ALIKE [2, 2, 2, 0, 2]
ALIVE [2, 2, 2, 2, 2]
Word Guessing: NOISE
AROSE [0, 0, 1, 2, 2]
DOUSE [0, 2, 0, 2,

...

YIELD [0, 2, 2, 1, 0]
LIETH [2, 2, 2, 2, 2]
Word Guessing: SOMAS
AROSE [1, 0, 1, 1, 0]
TAILS [0, 1, 0, 0, 2]
COUPS [0, 2, 0, 0, 2]
MOANS [1, 2, 1, 0, 2]
SOFAS [2, 2, 0, 2, 2]
SODAS [2, 2, 0, 2, 2]
Word Guessing: DARKY
AROSE [1, 1, 0, 0, 0]
INLAY [0, 0, 0, 1, 2]
PURTY [0, 0, 2, 0, 2]
BARMY [0, 2, 2, 0, 2]
HARRY [0, 2, 2, 0, 2]
DARKY [2, 2, 2, 2, 2]
Word Guessing: VILLI
AROSE [0, 0, 0, 0, 0]
UNITY [0, 0, 1, 0, 0]
MILCH [0, 2, 2, 0, 0]
VILLI [2, 2, 2, 2, 2]
Word Guessing: GLUON
AROSE [0, 0, 1, 0, 0]
UNITY [1, 1, 0, 0, 0]
MUCHO [0, 1, 0, 0, 1]
FOUND [0, 1, 2, 1, 0]
GLUON [2, 2, 2, 2, 2]
Word Guessing: JANES
AROSE [1, 0, 0, 1, 1]
TALES [0, 2, 0, 2, 2]
CAVES [0, 2, 0, 2, 2]
GAMES [0, 2, 0, 2, 2]
EASES [1, 2, 1, 2, 2]
SAKES [1, 2, 0, 2, 2]
Word Guessing: CANTS
AROSE [1, 0, 0, 1, 0]
TAILS [1, 2, 0, 0, 2]
PANTS [0, 2, 2, 2, 2]
WANTS [0, 2, 2, 2, 2]
CANTS [2, 2, 2, 2, 2]
Word Guessing: FARTS
AROSE [1, 1, 0, 1, 0]
TAILS [1, 2, 0, 0, 2]
RANTS [1, 2, 0, 2, 2]
FARTS [2, 2, 2, 2, 2]
Word Guessing: SO

ISLET [1, 1, 1, 1, 0]
PENIS [0, 2, 0, 1, 2]
DECKS [0, 2, 1, 0, 2]
SEEMS [1, 2, 0, 0, 2]
VEILS [0, 2, 2, 2, 2]
Word Guessing: IMMIX
AROSE [0, 0, 0, 0, 0]
UNITY [0, 0, 1, 0, 0]
MILCH [1, 1, 0, 0, 0]
IMMIX [2, 2, 2, 2, 2]
Word Guessing: YORES
AROSE [0, 1, 1, 1, 1]
OSIER [1, 1, 0, 2, 1]
DOLES [0, 2, 0, 2, 2]
ROPES [1, 2, 0, 2, 2]
NOSES [0, 2, 1, 2, 2]
SORES [1, 2, 2, 2, 2]
Word Guessing: DIEST
AROSE [0, 0, 0, 2, 1]
HEIST [0, 1, 1, 2, 2]
DIEST [2, 2, 2, 2, 2]
Word Guessing: BUBBA
AROSE [1, 0, 0, 0, 0]
INLAY [0, 0, 0, 1, 0]
MATCH [0, 1, 0, 0, 0]
BUFFA [2, 2, 0, 0, 2]
BUBBA [2, 2, 2, 2, 2]
Word Guessing: POMPS
AROSE [0, 0, 1, 1, 0]
LOUIS [0, 2, 0, 0, 2]
CONKS [0, 2, 0, 0, 2]
TOMBS [0, 2, 2, 0, 2]
POMPS [2, 2, 2, 2, 2]
Word Guessing: FORKY
AROSE [0, 1, 1, 0, 0]
UNITY [0, 0, 0, 0, 2]
LORDY [0, 2, 2, 0, 2]
PORKY [0, 2, 2, 2, 2]
CORKY [0, 2, 2, 2, 2]
FORKY [2, 2, 2, 2, 2]
Word Guessing: TURDY
AROSE [0, 1, 0, 0, 0]
UNITY [1, 0, 0, 1, 2]
LUMPY [0, 2, 0, 0, 2]
CURDY [0, 2, 2, 2, 2]
TURDY [2, 2, 2, 2

ALOFT [2, 0, 2, 0, 0]
AMONG [2, 0, 2, 0, 0]
AVOID [2, 0, 2, 2, 0]
AZOIC [2, 2, 2, 2, 2]
Word Guessing: ICERS
AROSE [0, 1, 0, 1, 1]
ILEUS [2, 0, 2, 0, 2]
ITEMS [2, 0, 2, 0, 2]
ICERS [2, 2, 2, 2, 2]
Word Guessing: SUERS
AROSE [0, 1, 0, 1, 1]
ILEUS [0, 0, 2, 1, 2]
WHETS [0, 0, 2, 0, 2]
DYERS [0, 0, 2, 2, 2]
SEERS [2, 1, 2, 2, 2]
SUERS [2, 2, 2, 2, 2]
Word Guessing: WIZEN
AROSE [0, 0, 0, 0, 1]
YIELD [0, 2, 1, 0, 0]
NIGHT [1, 2, 0, 0, 0]
MIXUP [0, 2, 0, 0, 0]
FINIF [0, 2, 1, 0, 0]
WIZEN [2, 2, 2, 2, 2]
Word Guessing: KOINE
AROSE [0, 0, 1, 0, 2]
GUILE [0, 0, 2, 0, 2]
CHINE [0, 0, 2, 2, 2]
OPINE [1, 0, 2, 2, 2]
TWINE [0, 0, 2, 2, 2]
KOINE [2, 2, 2, 2, 2]
Word Guessing: TOPOS
AROSE [0, 0, 1, 1, 0]
LOUIS [0, 2, 0, 0, 2]
CONKS [0, 2, 0, 0, 2]
TOMBS [2, 2, 0, 0, 2]
TOFFS [2, 2, 0, 0, 2]
TOPOS [2, 2, 2, 2, 2]
Word Guessing: SHIRR
AROSE [0, 1, 0, 1, 0]
UNITS [0, 0, 2, 0, 1]
SLICK [2, 0, 2, 0, 0]
SHIRR [2, 2, 2, 2, 2]
Word Guessing: RIFER
AROSE [0, 1, 0, 0, 1]
UTERI [0, 0, 1, 1, 1]
IDLER [1, 0, 0, 2

TAILS [0, 2, 0, 0, 2]
RANKS [1, 2, 0, 2, 2]
BACKS [0, 2, 0, 2, 2]
MASKS [0, 2, 1, 2, 2]
HARKS [2, 2, 2, 2, 2]
Word Guessing: WAZOO
AROSE [1, 0, 1, 0, 0]
INLAY [0, 0, 0, 1, 0]
MOCHA [0, 1, 0, 0, 1]
FAGOT [0, 2, 0, 2, 0]
KAZOO [0, 2, 2, 2, 2]
WAZOO [2, 2, 2, 2, 2]
Word Guessing: FENNY
AROSE [0, 0, 0, 0, 1]
YIELD [1, 0, 1, 0, 0]
UMPTY [0, 0, 0, 0, 2]
JENNY [0, 2, 2, 2, 2]
FENNY [2, 2, 2, 2, 2]
Word Guessing: NAVES
AROSE [1, 0, 0, 1, 1]
TALES [0, 2, 0, 2, 2]
CAVES [0, 2, 2, 2, 2]
WAVES [0, 2, 2, 2, 2]
SAVES [1, 2, 2, 2, 2]
EAVES [1, 2, 2, 2, 2]
Word Guessing: KEYER
AROSE [0, 1, 0, 0, 1]
UTERI [0, 0, 1, 1, 0]
REDLY [1, 2, 0, 0, 1]
WENCH [0, 2, 0, 0, 0]
FEVER [0, 2, 0, 2, 2]
KEYER [2, 2, 2, 2, 2]
Word Guessing: TUFAS
AROSE [1, 0, 0, 1, 0]
TAILS [2, 1, 0, 0, 2]
TUNAS [2, 2, 0, 2, 2]
TUBAS [2, 2, 0, 2, 2]
TUFAS [2, 2, 2, 2, 2]
Word Guessing: MORPH
AROSE [0, 1, 1, 0, 0]
UNITY [0, 0, 0, 0, 0]
PORCH [1, 2, 2, 0, 2]
MORPH [2, 2, 2, 2, 2]
Word Guessing: RAJAS
AROSE [1, 1, 0, 1, 0]
TAILS [0, 2, 0, 0

UNITY [0, 0, 1, 0, 2]
IMPLY [1, 0, 0, 0, 2]
DICKY [2, 2, 0, 0, 2]
DIDDY [2, 2, 2, 2, 2]
Word Guessing: SHMOO
AROSE [0, 0, 1, 1, 0]
LOUIS [0, 1, 0, 0, 1]
PSYCH [0, 1, 0, 0, 1]
OXBOW [1, 0, 0, 2, 0]
SHMOO [2, 2, 2, 2, 2]
Word Guessing: BRUSK
AROSE [0, 2, 0, 2, 0]
CRUST [0, 2, 2, 2, 0]
BRUSH [2, 2, 2, 2, 0]
BRUSK [2, 2, 2, 2, 2]
Word Guessing: PREST
AROSE [0, 2, 0, 2, 1]
CREST [0, 2, 2, 2, 2]
WREST [0, 2, 2, 2, 2]
PREST [2, 2, 2, 2, 2]
Word Guessing: CURER
AROSE [0, 1, 0, 0, 1]
UTERI [1, 0, 1, 1, 0]
MULEY [0, 2, 0, 2, 0]
CURED [2, 2, 2, 2, 0]
CURER [2, 2, 2, 2, 2]
Word Guessing: PASTS
AROSE [1, 0, 0, 1, 0]
TAILS [1, 2, 0, 0, 2]
PANTS [2, 2, 0, 2, 2]
PACTS [2, 2, 0, 2, 2]
PASTS [2, 2, 2, 2, 2]
Word Guessing: KELPY
AROSE [0, 0, 0, 0, 1]
YIELD [1, 0, 1, 1, 0]
MULEY [0, 0, 2, 1, 2]
KELPY [2, 2, 2, 2, 2]
Word Guessing: BOCCE
AROSE [0, 0, 1, 0, 2]
GUILE [0, 0, 0, 0, 2]
MONTE [0, 2, 0, 0, 2]
WOWEE [0, 2, 0, 1, 2]
BOCCE [2, 2, 2, 2, 2]
Word Guessing: KICKY
AROSE [0, 0, 0, 0, 0]
UNITY [0, 0, 1, 0,

In [25]:
answer_orig_wt_history.describe()

,answer,first_guess,second_guess,third_guess,fourth_guess,fifth_guess,sixth_guess,No Solution
count,5757,5757,5756,5638,4640,2776,1350,621
unique,5757,1,118,1028,1935,1457,737,1
top,WHICH,AROSE,TILES,CANES,DAMES,EASES,SAGES,1
freq,1,5757,454,59,43,38,32,621


In [27]:
answer_rewt_history.describe()

,answer,first_guess,second_guess,third_guess,fourth_guess,fifth_guess,sixth_guess,No Solution
count,5757,5757,5756,5690,5117,3556,2127,1021
unique,5757,1,75,760,1826,1630,1234,1
top,WHICH,AROSE,UNITY,LUMPY,GAMES,EASES,SAKES,1
freq,1,5757,641,127,52,51,36,1021


# Individual Word Tester

In [18]:
# establishing loop through database, testing on highest strength word
answer_orig_wt_history = pd.DataFrame(columns = ['answer', 'first_guess', 'second_guess', 'third_guess', 
                                   'fourth_guess', 'fifth_guess', 'sixth_guess', 'No Solution'])

answer_word = []
answer_char = []
word_played = []
word_played_char = []
word_played_result = []
result_char_count_temp = {}
answer_char_count_temp = {}

word = db[1]
print(word)
    
char_loc_ref_full()

answer_char.clear()
word_played.clear()
word_played_char.clear()
word_played_result.clear()
result_char_count_temp.clear()
answer_char_count_temp.clear()

# establishing round counter
rounds_played = 0

answer_word = word
answer_char = split(answer_word)

for char in answer_char:
    if (char in answer_char_count_temp):
        answer_char_count_temp[char] += 1
    else:
        answer_char_count_temp[char] = 1

answer_orig_wt_history.loc[len(answer_orig_wt_history.index), 'answer'] = answer_word

# running counter on original database           
char_totals_func(db)

# sorting for uniques on original data set
db_uniques_func(db)
word_st_func(db_uniques)

THERE


In [19]:
    while rounds_played <= 6:

        # determining word to play, splitting for characters, and adding to data set of answers
        word_played = max(word_st, key=word_st.get)
        word_played_char = split(word_played)
        word_played_result.clear()
        result_char_count_temp.clear()
        rounds_played += 1
        answer_orig_wt_history.iloc[len(answer_orig_wt_history.index)-1, [rounds_played]] = word_played

        # finding result from playing word
        for i in range(len(word_played)):
            if word_played_char[i] not in answer_char:
                word_played_result.append(0)

            elif word_played_char[i] == answer_char[i]:
                word_played_result.append(2)

                if (word_played_char[i] in result_char_count_temp):
                    result_char_count_temp[word_played_char[i]] += 1
                else:
                    result_char_count_temp[word_played_char[i]] = 1

            # do not need to check that it does not match as all matching letters will be handled above
            elif word_played_char[i] in answer_char:

                if word_played_char[i] in result_char_count_temp:
                    if result_char_count_temp[word_played_char[i]] == answer_char_count_temp[word_played_char[i]]:
                        word_played_result.append(0)
                        
                    else:
                        word_played_result.append(1) 
                        result_char_count_temp[word_played_char[i]] += 1

                else:
                    word_played_result.append(1)                   
                    result_char_count_temp[word_played_char[i]] = 1                    
            else:
                print("Error in assigning values to word_played_result based upon word played")
                break

        print(word_played, word_played_result)
        print(answer_char_count_temp, result_char_count_temp)

        if word_played_result == [2,2,2,2,2]:
            # fill unfilled round columns with placeholder (0?)
            print("Answer Met")
            break

        if rounds_played == 6:
            answer_orig_wt_history.iloc[len(answer_orig_wt_history.index)-1, 'No Solution'] = 1
            break

        else:

            # running filtering character reference list based on first round
            char_loc_ref_filt_func(word_played, word_played_result)

            # filtered data set based upon results of round
            if rounds_played == 1:
                db_filtered_func(db)
                print(db_filtered)
            else:
                db_filtered_func(db_filtered)
                print(db_filtered)

        # determing distribution of letters, based upon filtered data set, if reweight = 'True' determining new letter weighting

        char_totals_func(db_filtered, reweight = False)
        print(char_totals)

        # filtering for words with five unique letters on filtered data set for first three rounds
        if rounds_played <= 3 and len(db_filtered) != 1:
            db_uniques_func(db_filtered)
            
            if bool(db_uniques):
                word_st_func(db_uniques)
                print("db_uniques")
                print(db_uniques)
            
            else:
                word_st_func(db_filtered)
                print("db_filtered")
                print(db_filtered)

        else:
            word_st_func(db_filtered)
            print("db_filtered")
            print(db_filtered)

AROSE [0, 1, 0, 0, 2]
{'T': 1, 'H': 1, 'E': 2, 'R': 1} {'R': 1, 'E': 1}
['THERE', 'WHERE', 'THREE', 'WHILE', 'WHITE', 'QUITE', 'PIECE', 'TWICE', 'GUIDE', 'TITLE', 'KNIFE', 'FENCE', 'JUDGE', 'CURVE', 'THEME', 'HENCE', 'RHYME', 'UNCLE', 'JUICE', 'RIFLE', 'CYCLE', 'NERVE', 'RIDGE', 'FLUTE', 'LEDGE', 'BUGLE', 'EXILE', 'HEDGE', 'UNITE', 'WEDGE', 'GLIDE', 'NIECE', 'EERIE', 'BUDGE', 'FUDGE', 'THINE', 'HINGE', 'BULGE', 'TWINE', 'NUDGE', 'CHUTE', 'UNTIE', 'BEIGE', 'UNDUE', 'MERGE', 'GENIE', 'WHINE', 'NICHE', 'PURGE', 'PLUME', 'TITHE', 'MINCE', 'GUILE', 'TEPEE', 'LUNGE', 'LITHE', 'VERGE', 'QUEUE', 'PENCE', 'BIBLE', 'THYME', 'CHIME', 'WINCE', 'BUTTE', 'CHIDE', 'TINGE', 'LEVEE', 'ELUDE', 'DIRGE', 'DELVE', 'ETUDE', 'DUNCE', 'PIXIE', 'PEWEE', 'FUGUE', 'ELITE', 'GENRE', 'FLUKE', 'LEMME', 'DUPLE', 'REVUE', 'VERVE', 'TILDE', 'BELLE', 'MEBBE', 'PIQUE', 'RUPEE', 'GIMME', 'QUIRE', 'EYRIE', 'FIBRE', 'BILGE', 'PUREE', 'TULLE', 'CHIVE', 'METRE', 'MELEE', 'BINGE', 'CUTIE', 'CHINE', 'VENUE', 'LIEGE', 'EMCEE', 

# Individual word tester, individual rounds

In [ ]:
# establishing loop through database, testing on highest strength word
answer_orig_wt_history = pd.DataFrame(columns = ['answer', 'first_guess', 'second_guess', 'third_guess', 
                                   'fourth_guess', 'fifth_guess', 'sixth_guess', 'No Solution'])

answer_word = []
answer_char = []
word_played = []
word_played_char = []
word_played_result = []
result_char_count_temp = {}
answer_char_count_temp = {}

word = db[0]
print(word)
    
char_loc_ref_full()

answer_char.clear()
word_played.clear()
word_played_char.clear()
word_played_result.clear()
result_char_count_temp.clear()
answer_char_count_temp.clear()

# establishing round counter
rounds_played = 0

answer_word = word
answer_char = split(answer_word)

for char in answer_char:
    if (char in answer_char_count_temp):
        answer_char_count_temp[char] += 1
    else:
        answer_char_count_temp[char] = 1

answer_orig_wt_history.loc[len(answer_orig_wt_history.index), 'answer'] = answer_word

# running counter on original database           
char_totals_func(db)

# sorting for uniques on original data set
db_uniques_func(db)
word_st_func(db_uniques)

In [ ]:
    # determining word to play, splitting for characters, and adding to data set of answers
    word_played = max(word_st, key=word_st.get)
    word_played_char = split(word_played)
    word_played_result.clear()
    result_char_count_temp.clear()
    rounds_played += 1
    answer_orig_wt_history.iloc[len(answer_orig_wt_history.index)-1, [rounds_played]] = word_played

    # finding result from playing word
    for i in range(len(word_played)):
        if word_played_char[i] not in answer_char:
            word_played_result.append(0)

        elif word_played_char[i] == answer_char[i]:
            word_played_result.append(2)

            if (word_played_char[i] in result_char_count_temp):
                result_char_count_temp[word_played_char[i]] += 1
            else:
                result_char_count_temp[word_played_char[i]] = 1

        # do not need to check that it does not match as all matching letters will be handled above
        elif word_played_char[i] in answer_char:

                if word_played_char[i] in result_char_count_temp:
                    if result_char_count_temp[word_played_char[i]] == answer_char_count_temp[word_played_char[i]]:
                        word_played_result.append(0)

                    else:
                        word_played_result.append(1) 
                        result_char_count_temp[word_played_char[i]] += 1

            else:
                word_played_result.append(1)                   
                result_char_count_temp[word_played_char[i]] = 1                    
        else:
            print("Error in assigning values to word_played_result based upon word played")
            break

    print(word_played, word_played_result)
    print(answer_char_count_temp, result_char_count_temp)

    if word_played_result == [2,2,2,2,2]:
        # fill unfilled round columns with placeholder (0?)
        print("Answer Met")

    if rounds_played == 6:
        answer_orig_wt_history.iloc[len(answer_orig_wt_history.index)-1, 'No Solution'] = 1

    else:

        # running filtering character reference list based on first round
        char_loc_ref_filt_func(word_played, word_played_result)

        # filtered data set based upon results of round
        if rounds_played == 1:
            db_filtered_func(db)
        else:
            db_filtered_func(db_filtered)

    # determing distribution of letters, based upon filtered data set, if reweight = 'True' determining new letter weighting

    char_totals_func(db_filtered, reweight = False)

    # filtering for words with five unique letters on filtered data set for first three rounds
    if rounds_played <= 3 and len(db_filtered) != 1:
        db_uniques_func(db_filtered)

        if bool(db_uniques):
            word_st_func(db_uniques)
            print("db_uniques")
            print(db_uniques)

        else:
            word_st_func(db_filtered)
            print("db_filtered")
            print(db_filtered)

    else:
        word_st_func(db_filtered)
        print("db_filtered")
        print(db_filtered)